In [1]:
import os
import platform
import sys

def get_project_root() -> str:
    system = platform.system()
    return r"D:\Users\tcshore\Documents\Projects\Tangrams\Data" if system == "Windows" else "/home/tshore/Projects/tangrams-restricted/Data"

infile_path = os.path.join(get_project_root(), "wordprobseqs-inflected-small.tsv")
print("Will read file \"{}\".".format(infile_path), file=sys.stderr)

Will read file "D:\Users\tcshore\Documents\Projects\Tangrams\Data\wordprobseqs-inflected-small.tsv".


In [2]:
import csv
import pandas as pd

RESULTS_FILE_CSV_DIALECT = csv.excel_tab
# NOTE: "category" dtype doesn't work with pandas-0.21.0 but does with pandas-0.21.1
__RESULTS_FILE_DTYPES = {"DYAD": "category", "WORD": "category", "IS_TARGET": bool, "IS_OOV": bool,
				 "IS_INSTRUCTOR": bool, "SHAPE": "category", "ONLY_INSTRUCTOR": bool, "WEIGHT_BY_FREQ": bool}

def read_results_file(inpath: str, encoding: str) -> pd.DataFrame:
	print("Reading \"{}\" using encoding \"{}\".".format(inpath, encoding), file=sys.stderr)
	result = pd.read_csv(inpath, dialect=RESULTS_FILE_CSV_DIALECT, sep=RESULTS_FILE_CSV_DIALECT.delimiter,
						 float_precision="round_trip",
						 encoding=encoding, memory_map=True, dtype=__RESULTS_FILE_DTYPES)
	return result

cv_results = read_results_file(infile_path, "windows-1252")
print("Read {} cross-validation results for {} dyad(s).".format(cv_results.shape[0], cv_results["DYAD"].nunique()),
      file=sys.stderr)

Reading "D:\Users\tcshore\Documents\Projects\Tangrams\Data\wordprobseqs-inflected-small.tsv" using encoding "windows-1252".
Read 180880 cross-validation results for 2 dyad(s).


In [3]:
def find_target_ref_rows(df: pd.DataFrame) -> pd.DataFrame:
	result = df.loc[df["IS_TARGET"] == True]
	result_row_count = result.shape[0]
	complement_row_count = df.loc[~df.index.isin(result.index)].shape[0]
	assert result_row_count + complement_row_count == df.shape[0]
	print("Found {} nontarget rows and {} target rows. Ratio: {}".format(complement_row_count, result_row_count,
																		 complement_row_count / float(
																			 result_row_count)), file=sys.stderr)
	return result

cv_results = find_target_ref_rows(cv_results)

Found 171836 nontarget rows and 9044 target rows. Ratio: 19.0


In [6]:
# Create vocab before splitting training and testing DFs so that the word feature set is stable
vocab = tuple(sorted(cv_results["WORD"].unique()))
print("Fitting one-hot encoder for vocabulary of size {}.".format(len(vocab)), file=sys.stderr)
# https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
print("Creating vocab dictionary for one-hot label encoding.", file=sys.stderr)
vocab_idxs = dict((word, idx) for (idx, word) in enumerate(vocab))

Fitting one-hot encoder for vocabulary of size 299.
Creating vocab dictionary for one-hot label encoding.


In [7]:
from typing import Tuple

import numpy as np

def split_training_testing(df: pd.DataFrame, test_set_size : int) -> Tuple[pd.DataFrame, pd.DataFrame]:
	dyad_ids = df["DYAD"].unique()
	training_set_size = len(dyad_ids) - test_set_size
	if training_set_size < 1:
		raise ValueError("Desired test set size is {} but only {} dyads found.".format(test_set_size, len(dyad_ids)))
	else:
		training_set_dyads = frozenset(np.random.choice(dyad_ids, training_set_size))
		print("Training set dyads: {}".format(sorted(training_set_dyads)), file=sys.stderr)
		training_set_idxs = df["DYAD"].isin(training_set_dyads)
		training_set = df.loc[training_set_idxs]
		test_set = df.loc[~training_set_idxs]
		test_set_dyads = frozenset(test_set["DYAD"].unique())
		print("Test set dyads: {}".format(sorted(test_set_dyads)), file=sys.stderr)

		assert not frozenset(training_set["DYAD"].unique()).intersection(frozenset(test_set_dyads))
		return training_set, test_set
    
training_df, test_df = split_training_testing(cv_results, 1)
training_df = training_df.copy(deep=False)
test_df = test_df.copy(deep=False)

Training set dyads: ['17']
Test set dyads: ['1']


In [10]:
from typing import Iterator, List, Mapping

class SequenceMatrixFactory(object):

	def __init__(self,vocab_idxs : Mapping[str, int]):
		self.__vocab_idxs = vocab_idxs
		self.__feature_count = len(self.__vocab_idxs) + 3

	def create_datapoint_feature_array(self, row: pd.Series) -> List[float]:
		word_features = [0.0] * len(self.__vocab_idxs)
		# The features representing each individual vocabulary word are at the beginning of the feature vector
		word_features[self.__vocab_idxs[row["WORD"]]] = 1.0
		is_instructor = 1.0 if row["IS_INSTRUCTOR"] else 0.0
		is_oov = 1.0 if row["IS_OOV"] else 0.0
		#is_target = 1.0 if row["IS_TARGET"] else 0.0
		score = row["PROBABILITY"]
		other_features = list((is_instructor, is_oov, score))
		result = word_features + other_features
		#print("Created a vector of {} features.".format(len(result)), file=sys.stderr)
		return result

	def __call__(self, df : pd.DataFrame) -> np.array:
		# https://stackoverflow.com/a/47815400/1391325
		df.sort_values("TOKEN_SEQ_ORDINALITY", inplace=True)
		sequence_groups = df.groupby(("CROSS_VALIDATION_ITER", "DYAD", "SPLIT_SEQ_NO", "UTT_START_TIME", "UTT_END_TIME", "ENTITY"), as_index=False)
		return np.array(tuple(tuple(self.__create_feature_vectors(seq)) for _, seq in sequence_groups))
        
	def __create_feature_vectors(self, df : pd.DataFrame) -> Iterator[List[float]]:
		# noinspection PyProtectedMember
		return (self.create_datapoint_feature_array(row._asdict()) for row in df.itertuples(index=False))

print("Splitting token sequences.", file=sys.stderr)
seq_matrix_factory = SequenceMatrixFactory(vocab_idxs)
training_matrix = seq_matrix_factory(training_df)
print("Created a training data matrix of shape {}.".format(training_matrix.shape), file=sys.stderr)
test_matrix = seq_matrix_factory(test_df)
print("Created a test data matrix of shape {}.".format(test_matrix.shape), file=sys.stderr)

Splitting token sequences.
Created a training data matrix of shape (1174, 4, 302).
Created a test data matrix of shape (1087, 4, 302).


In [13]:
training_x = training_matrix[:,:,:-1]
print(training_x.shape)
assert len(training_x.shape) == 3
training_y = training_matrix[:,:,-1]
print(training_y.shape)
assert len(training_y.shape) == 2

from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential

model = Sequential()
#word_embeddings = Embedding(len(vocab), embedding_vector_length, input_length=max_review_length)
#model.add(word_embeddings)
# model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
# input shape is a pair of (timesteps, features) <https://stackoverflow.com/a/44583784/1391325>
input_shape = training_x.shape[1:3]
print("Input shape: {}".format(input_shape), file=sys.stderr)
units = training_y.shape[1]
print("Units: {}".format(units), file=sys.stderr)
lstm = LSTM(input_shape=input_shape, units=units)
#lstm = LSTM(batch_input_shape = training_x.shape, stateful = True, units=len(training_y.shape))
model.add(lstm)
model.add(Dense(units, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

(1174, 4, 301)
(1174, 4)


Input shape: (4, 301)
Units: 4


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 4)                 4896      
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 20        
Total params: 4,916
Trainable params: 4,916
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
# train LSTM
#model.fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
#training_eval = model.fit(x=training_x, y=training_y, epochs=1, verbose=1)

# train LSTM <https://machinelearningmastery.com/memory-in-a-long-short-term-memory-network/>
epochs = 5
for i in range(epochs):
    x = training_x[i]
    x_oldshape = x.shape
    x_newshape = (-1, x_oldshape[0], x_oldshape[1])
    x = np.reshape(x, x_newshape)
    #print("x.shape = {}".format(x.shape), file=sys.stderr)
    y = training_y[i]
    y_oldshape = y.shape
    y_newshape = (-1, y_oldshape[0])
    y = np.reshape(y, y_newshape)
    #print("y.shape = {}".format(y.shape), file=sys.stderr)
    model.fit(x, y, epochs=1, batch_size=1, verbose=1, shuffle=False)
    model.reset_states()

Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.6843 - acc: 0.5000
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.6830 - acc: 0.5000
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.6812 - acc: 0.5000
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.6843 - acc: 0.5000
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.6841 - acc: 0.5000
